In [89]:
import pandas as pd

### Active buyers preprocessing

In [1]:
active_buyers = pd.read_csv('data/raw/active_buyers.csv')
active_buyers.head()

NameError: name 'pd' is not defined

In [140]:
active_buyers.shape

(3848023, 14)

In [141]:
active_buyers.isnull().sum()

mbr_lic_type                        14688
mbr_state                               0
lot_nbr                                 0
buyer_nbr                               0
mbr_email                               0
max_bid                                 0
inv_dt                                  0
lot_year                                0
lot_make_cd                             0
grp_model                          103036
acv                                     0
repair_cost                             0
total_unique_buyers_on_that_lot         0
total_unique_lots_bid_by_buyers         0
dtype: int64

### Missing value imputations based on:
1. mbr lic type: Mode
2. mbr state: Mode
3. grp model : Mode within year and make

In [142]:
active_buyers['mbr_lic_type'] = active_buyers['mbr_lic_type'].fillna(active_buyers['mbr_lic_type'].mode()[0])
active_buyers['mbr_state'] = active_buyers['mbr_state'].fillna(active_buyers['mbr_state'].mode()[0])

In [143]:
## Grp model missing impute
active_buyers[active_buyers['grp_model'].isna()]

,mbr_lic_type,mbr_state,lot_nbr,buyer_nbr,mbr_email,max_bid,inv_dt,lot_year,lot_make_cd,grp_model,acv,repair_cost,total_unique_buyers_on_that_lot,total_unique_lots_bid_by_buyers
48596,Export,DE,71856465,96045,mohamed@edfa3ly.com,10700.0,2025-09-15,2024,NISS,NaN,22404.00,0.00,19,191
48664,Consumer,NV,82087935,341672,randyscott404@gmail.com,2050.0,2025-09-29,2023,PLSR,NaN,26977.00,18310.20,3,9
48676,Dismantler,NV,66676435,39955,luke@nvpap.com,125.0,2025-08-06,2024,NISS,NaN,21611.00,0.00,5,5421
48707,Export,DE,53662215,96045,mohamed@edfa3ly.com,5400.0,2025-07-15,2023,KIA,NaN,20891.00,12267.51,7,191
48864,Dismantler,NV,64113785,39955,luke@nvpap.com,125.0,2025-09-12,2025,TOYT,NaN,29425.00,34401.83,6,5421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3727644,Dealer,WI,67201425,580353,cgilroy121@gmail.com,275.0,2025-08-19,2025,NISS,NaN,0.00,0.00,10,3093
3727658,Export,WI,67966295,634787,43678707-MSTRBDR-83993959-1957-4718-ad43-fc7d6...,46500.0,2025-09-10,2024,LEXS,NaN,63746.00,47418.51,9,1337
3727672,Export,WI,65513095,706229,127264-MSTRBDR-32763034-9843-4609-af28-0814596...,12100.0,2025-08-11,2022,VOLV,NaN,40361.00,36174.17,3,2000
3727685,Dealer,WI,65502855,824620,prestigeauto6767@gmail.com,6600.0,2025-08-14,2024,FORD,NaN,29586.00,27449.16,5,1157


In [144]:
# Step 1: Fill missing grp_model using mode within (lot_year, lot_make_cd)
def fill_grp_model_year_make(group):
    mode_val = group['grp_model'].mode()
    if not mode_val.empty:
        group['grp_model'] = group['grp_model'].fillna(mode_val[0])
    return group

active_buyers = active_buyers.groupby(['lot_year', 'lot_make_cd'], group_keys=False).apply(fill_grp_model_year_make)

# Step 2: Fill remaining missing grp_model using mode within lot_make_cd only
def fill_grp_model_make(group):
    mode_val = group['grp_model'].mode()
    if not mode_val.empty:
        group['grp_model'] = group['grp_model'].fillna(mode_val[0])
    return group

active_buyers = active_buyers.groupby('lot_make_cd', group_keys=False).apply(fill_grp_model_make)


/var/folders/jj/1fql_rw17h10wm_w98gwx9wm0000gq/T/ipykernel_22311/2787800539.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  active_bidders = active_bidders.groupby(['lot_year', 'lot_make_cd'], group_keys=False).apply(fill_grp_model_year_make)
/var/folders/jj/1fql_rw17h10wm_w98gwx9wm0000gq/T/ipykernel_22311/2787800539.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  active_bidders = active_bidders.gr

In [145]:
active_buyers.isnull().sum()

mbr_lic_type                          0
mbr_state                             0
lot_nbr                               0
buyer_nbr                             0
mbr_email                             0
max_bid                               0
inv_dt                                0
lot_year                              0
lot_make_cd                           0
grp_model                          2020
acv                                   0
repair_cost                           0
total_unique_buyers_on_that_lot       0
total_unique_lots_bid_by_buyers       0
dtype: int64

In [146]:
active_buyers[active_buyers['grp_model'].isna()]

,mbr_lic_type,mbr_state,lot_nbr,buyer_nbr,mbr_email,max_bid,inv_dt,lot_year,lot_make_cd,grp_model,acv,repair_cost,total_unique_buyers_on_that_lot,total_unique_lots_bid_by_buyers
48664,Consumer,NV,82087935,341672,randyscott404@gmail.com,2050.0,2025-09-29,2023,PLSR,NaN,26977.00,18310.20,3,9
49122,Dealer,DE,64814695,6088,info@ram-shipping.com,6200.0,2025-10-02,2024,VNFS,NaN,31542.00,12815.62,6,769
58987,Dealer,NH,53683435,527331,MYPLATINUMCAR@GMAIL.COM,19800.0,2025-07-17,2023,RIVA,NaN,81175.66,0.00,15,333
66879,General Business,NV,59657985,177825,urusciuc@yahoo.com,1950.0,2025-08-12,2022,LUCI,NaN,66285.00,71210.06,11,2
92759,Dealer,NV,53683435,36692,bguido1@hotmail.com,8500.0,2025-07-17,2023,RIVA,NaN,81175.66,0.00,15,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716304,Dealer,WI,57598465,179180,euroresourceauto@gmail.com,12600.0,2025-08-15,2023,RIVA,NaN,75252.00,0.00,9,1033
3726216,Dealer,WI,70345045,580353,cgilroy121@gmail.com,225.0,2025-09-23,2024,MAHI,NaN,26033.00,18544.45,11,3093
3726693,Consumer,WI,57997725,653753,20016525-MSTRBDR-e246d791-0af5-439f-83fc-926d0...,3650.0,2025-08-12,2023,PLSR,NaN,34754.00,40700.10,3,250
3726829,Export,WI,59595425,634787,43678707-MSTRBDR-83993959-1957-4718-ad43-fc7d6...,7100.0,2025-08-15,2023,LUCI,NaN,56920.00,56920.00,14,1337


In [147]:
## drop these lots with missing lot make
active_buyers.dropna(subset=['grp_model'], inplace=True)

In [148]:
active_buyers.isnull().sum()

mbr_lic_type                       0
mbr_state                          0
lot_nbr                            0
buyer_nbr                          0
mbr_email                          0
max_bid                            0
inv_dt                             0
lot_year                           0
lot_make_cd                        0
grp_model                          0
acv                                0
repair_cost                        0
total_unique_buyers_on_that_lot    0
total_unique_lots_bid_by_buyers    0
dtype: int64

In [149]:
active_buyers['buyer_nbr'].nunique()

80863

### Splitting <7 and >=7

In [150]:
data_high = active_buyers[active_buyers['total_unique_lots_bid_by_buyers']>=7]
data_low = active_buyers[active_buyers['total_unique_lots_bid_by_buyers']<7]
print(data_high['buyer_nbr'].nunique())
print(data_low['buyer_nbr'].nunique())

33242
47621


In [151]:
def even_odd_split_df(df, buyer_col='buyer_nbr'):
    # Extract last digit
    df = df.copy()
    df['last_digit'] = df[buyer_col] % 10

    # Split based on even-odd
    holdout_df = df[df['last_digit'] % 2 == 0].drop(columns='last_digit')
    test_df = df[df['last_digit'] % 2 != 0].drop(columns='last_digit')

    return holdout_df, test_df

# Apply to both dataframes
data_high_holdout, data_high_test = even_odd_split_df(data_high)
data_low_holdout, data_low_test = even_odd_split_df(data_low)

# Optional: Check counts
print(f"High Holdout Buyers: {data_high_holdout['buyer_nbr'].nunique()}")
print(f"High Test Buyers: {data_high_test['buyer_nbr'].nunique()}")
print(f"Low Holdout Buyers: {data_low_holdout['buyer_nbr'].nunique()}")
print(f"Low Test Buyers: {data_low_test['buyer_nbr'].nunique()}")

High Holdout Buyers: 16733
High Test Buyers: 16509
Low Holdout Buyers: 23883
Low Test Buyers: 23738


In [152]:
data_low_holdout.to_csv('data/interim/data_low_holdout.csv',index=False)
data_high_holdout.to_csv('data/interim/data_high_holdout.csv',index=False)

data_high_test.to_csv('data/interim/data_high_test.csv',index=False)
data_low_test.to_csv('data/interim/data_low_test.csv',index=False)

### Not Active buyers preprocessing

In [153]:
not_active_buyers = pd.read_csv('data/raw/not_active_buyers.csv')
not_active_buyers.head()

,mbr_lic_type,mbr_state,mbr_nbr,mbr_email
0,Consumer,SD,825681,jamestoney3@hotmail.com
1,Consumer,DE,645351,Rockdjr316m@aol.com
2,Consumer,AK,517624,zephy907@gmail.com
3,Consumer,NH,776575,suroyanehemiah@gmail.com
4,Export,PR,591581,cfuentes1091@gmail.com


In [154]:
not_active_buyers.isnull().sum()

mbr_lic_type    878
mbr_state         0
mbr_nbr           0
mbr_email         0
dtype: int64

In [155]:
not_active_buyers['mbr_lic_type'].fillna(not_active_buyers['mbr_lic_type'].mode()[0], inplace=True)

/var/folders/jj/1fql_rw17h10wm_w98gwx9wm0000gq/T/ipykernel_22311/3566338129.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  not_active_bidders['mbr_lic_type'].fillna(not_active_bidders['mbr_lic_type'].mode()[0], inplace=True)


In [156]:
not_active_buyers.isnull().sum()

mbr_lic_type    0
mbr_state       0
mbr_nbr         0
mbr_email       0
dtype: int64

In [157]:
not_active_buyers['mbr_nbr'].nunique()

145395

In [158]:
def even_odd_split_df(df, buyer_col='mbr_nbr'):
    # Extract last digit
    df = df.copy()
    df['last_digit'] = df[buyer_col] % 10

    # Split based on even-odd
    holdout_df = df[df['last_digit'] % 2 == 0].drop(columns='last_digit')
    test_df = df[df['last_digit'] % 2 != 0].drop(columns='last_digit')

    return holdout_df, test_df

In [159]:
not_active_buyers_holdout, not_active_buyers_test = even_odd_split_df(not_active_buyers)

In [160]:
print(f"Group 1 shape: {not_active_buyers_test.shape}")
print(f"Group 2 shape: {not_active_buyers_holdout.shape}")

Group 1 shape: (72732, 4)
Group 2 shape: (72663, 4)


In [161]:
not_active_buyers_test['mbr_nbr'].nunique()

72732

In [162]:
not_active_buyers_holdout.to_csv('data/interim/not_active_buyers_holdout.csv',index=False)
not_active_buyers_test.to_csv('data/interim/not_active_buyers_test.csv',index=False)

### Popular lots preprocess

In [163]:
popular_lots = pd.read_csv('data/raw/popular_lots.csv')

In [164]:
popular_lots.head()

,buyer_type,mbr_state,lot_make_cd,grp_model,cnt,median_acv,median_repair_cost,model_rank,rank
0,NaN,TX,NISS,SENTRA,4,12869.00,0.00,1,1
1,NaN,TX,JEP,WRANGLER,2,26515.11,11974.95,1,2
2,NaN,TX,BMW,4 SERIES,1,26481.00,25033.87,1,3
3,NaN,TX,HOND,CRV,1,38924.00,0.00,1,4
4,NaN,TX,FORD,FOCUS,1,7519.00,6124.27,1,5


In [165]:
popular_lots['buyer_type'].value_counts()

buyer_type
Dealer                         306
Consumer                       294
Dismantler                     269
General Business               199
Export                         160
Automotive Related Business     48
Name: count, dtype: int64

In [166]:
popular_lots['buyer_type'] = popular_lots['buyer_type'].replace('Automotive Related Business', 'General Business')

In [167]:
popular_lots['buyer_type'].value_counts()

buyer_type
Dealer              306
Consumer            294
Dismantler          269
General Business    247
Export              160
Name: count, dtype: int64

In [168]:
popular_lots.isnull().sum()

buyer_type            42
mbr_state              0
lot_make_cd            0
grp_model             27
cnt                    0
median_acv             0
median_repair_cost     0
model_rank             0
rank                   0
dtype: int64

In [169]:
popular_lots['buyer_type'] = popular_lots['buyer_type'].fillna(popular_lots['buyer_type'].mode()[0])

In [170]:
popular_lots = popular_lots.groupby('lot_make_cd', group_keys=False).apply(fill_grp_model_make)


/var/folders/jj/1fql_rw17h10wm_w98gwx9wm0000gq/T/ipykernel_22311/3628516480.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  popular_lots = popular_lots.groupby('lot_make_cd', group_keys=False).apply(fill_grp_model_make)


In [171]:
popular_lots.isnull().sum()

buyer_type            0
mbr_state             0
lot_make_cd           0
grp_model             1
cnt                   0
median_acv            0
median_repair_cost    0
model_rank            0
rank                  0
dtype: int64

In [172]:
popular_lots.dropna(subset=['grp_model'], inplace=True)

In [173]:
popular_lots.isnull().sum()

buyer_type            0
mbr_state             0
lot_make_cd           0
grp_model             0
cnt                   0
median_acv            0
median_repair_cost    0
model_rank            0
rank                  0
dtype: int64

In [174]:
popular_lots[(popular_lots['buyer_type']=='Dealer') & (popular_lots['mbr_state']=='TX')]

,buyer_type,mbr_state,lot_make_cd,grp_model,cnt,median_acv,median_repair_cost,model_rank,rank
0,Dealer,TX,NISS,SENTRA,4,12869.00,0.00,1,1
1,Dealer,TX,JEP,WRANGLER,2,26515.11,11974.95,1,2
2,Dealer,TX,BMW,4 SERIES,1,26481.00,25033.87,1,3
3,Dealer,TX,HOND,CRV,1,38924.00,0.00,1,4
4,Dealer,TX,FORD,FOCUS,1,7519.00,6124.27,1,5
5,Dealer,TX,AUDI,Q7,1,0.00,0.00,1,6
330,Dealer,TX,CHEV,SILVERADO,561,28425.00,18017.62,1,1
331,Dealer,TX,NISS,SENTRA,424,14330.00,8758.49,1,2
332,Dealer,TX,TOYT,CAMRY,418,20105.50,12140.35,1,3
333,Dealer,TX,CHEV,MALIBU,377,13438.97,7603.00,1,4


In [175]:
popular_lots = popular_lots[popular_lots['rank']<=6]

In [176]:
popular_lots[(popular_lots['buyer_type']=='Dealer') & (popular_lots['mbr_state']=='TX')]

,buyer_type,mbr_state,lot_make_cd,grp_model,cnt,median_acv,median_repair_cost,model_rank,rank
0,Dealer,TX,NISS,SENTRA,4,12869.00,0.00,1,1
1,Dealer,TX,JEP,WRANGLER,2,26515.11,11974.95,1,2
2,Dealer,TX,BMW,4 SERIES,1,26481.00,25033.87,1,3
3,Dealer,TX,HOND,CRV,1,38924.00,0.00,1,4
4,Dealer,TX,FORD,FOCUS,1,7519.00,6124.27,1,5
5,Dealer,TX,AUDI,Q7,1,0.00,0.00,1,6
330,Dealer,TX,CHEV,SILVERADO,561,28425.00,18017.62,1,1
331,Dealer,TX,NISS,SENTRA,424,14330.00,8758.49,1,2
332,Dealer,TX,TOYT,CAMRY,418,20105.50,12140.35,1,3
333,Dealer,TX,CHEV,MALIBU,377,13438.97,7603.00,1,4


In [177]:
popular_lots.to_csv('data/interim/popular_lots.csv',index=False)

### Upcoming lots

In [178]:
future_lots = pd.read_csv('data/raw/future_lots.csv')

In [179]:
future_lots.head()

,lot_nbr,lot_year,lot_make_cd,grp_model,damage_type_desc,repair_cost,acv,auc_dt,proquote_amt
0,71239515,2017,VOLK,PASSAT,FRONT END,9251.75,9841.73,2025-10-14,2544.05
1,81929665,2013,HYUN,VELOSTER,FRONT END,0.00,4250.00,2025-10-14,1225.38
2,67987075,2016,SUBA,CROSSTREK,FRAME DAMAGE,6898.95,7043.00,2025-10-15,1722.39
3,81834465,2012,CADI,CTS,FRONT END,0.00,608.00,2025-10-16,503.58
4,65324015,2014,CHEV,SONIC,FRONT END,3261.53,3927.00,2025-10-15,679.09


In [180]:
future_lots.isnull().sum()

lot_nbr               0
lot_year              0
lot_make_cd           0
grp_model           967
damage_type_desc      1
repair_cost           0
acv                   0
auc_dt                0
proquote_amt          0
dtype: int64

In [181]:
future_lots['damage_type_desc'] = future_lots['damage_type_desc'].fillna(future_lots['damage_type_desc'].mode()[0])

In [182]:
future_lots = future_lots.groupby(['lot_year', 'lot_make_cd'], group_keys=False).apply(fill_grp_model_year_make)


future_lots = future_lots.groupby('lot_make_cd', group_keys=False).apply(fill_grp_model_make)


/var/folders/jj/1fql_rw17h10wm_w98gwx9wm0000gq/T/ipykernel_22311/42068045.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  future_lots = future_lots.groupby(['lot_year', 'lot_make_cd'], group_keys=False).apply(fill_grp_model_year_make)
/var/folders/jj/1fql_rw17h10wm_w98gwx9wm0000gq/T/ipykernel_22311/42068045.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  future_lots = future_lots.groupby('lot_make_c

In [183]:
future_lots.isnull().sum()

lot_nbr              0
lot_year             0
lot_make_cd          0
grp_model           27
damage_type_desc     0
repair_cost          0
acv                  0
auc_dt               0
proquote_amt         0
dtype: int64

In [184]:
future_lots.dropna(subset=['grp_model'], inplace=True)

In [185]:
future_lots.isnull().sum()

lot_nbr             0
lot_year            0
lot_make_cd         0
grp_model           0
damage_type_desc    0
repair_cost         0
acv                 0
auc_dt              0
proquote_amt        0
dtype: int64

In [186]:
future_lots.to_csv('data/interim/future_lots.csv',index=False)